# <center> 데이터분석캡스톤디자인 </center>

### <center> 소셜 데이터를 활용한 텍스트 분석: 인스타그램 기반 검색 키워드 관련 트렌드 제공 </center>


<div style = "text-align: right"> 2022년 1학기 </div>

<div style = "text-align: right"> 지도 교수: 이원희 교수님 </div>

<div style = "text-align: right"> 팀 C: 2018100900 모준우, 2020103944 온유나, 2018100922 이승건 </div>


---

#### 필요 모듈 가져오기

In [ ]:
from bs4 import BeautifulSoup
from collections import Counter
from collections import defaultdict
import gensim
from gensim import corpora
from gensim import models
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.models import TfidfModel
from gensim.models.word2vec import Word2Vec
from gensim.models.wrappers import LdaMallet
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from nltk import FreqDist
import numpy as np
import os
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import re
from selenium import webdriver
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score
import smart_open
import time
import warnings
from wordcloud import WordCloud 
from wordcloud import STOPWORDS

#### 함수 정의: 한글인지, 아닌지 확인

In [ ]:
def isKoreanIncluded(word):
        for i in word:
            if ord(i) > int('0x1100',16) and ord(i) < int('0x11ff',16) :
                return True
            if ord(i) > int('0x3131',16) and ord(i) < int('0x318e',16) :
                return True
            if ord(i) > int('0xa960',16) and ord(i) < int('0xa97c',16) :
                return True
            if ord(i) > int('0xac00',16) and ord(i) < int('0xd7a3',16) :
                return True
            if ord(i) > int('0xd7b0',16) and ord(i) < int('0xd7fb',16) :
                return True

        return False

____
# Crawling

#### 함수 정의: 검색 키워드의 페이지를 반환

In [ ]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

#### 함수 정의: 검색 페이지에서 첫번째 게시글을 클릭

In [ ]:
def select_first(driver):
    first = driver.find_elements_by_css_selector("div._aagw")[0]
    print(type(first))
    first.click()
    time.sleep(3)

#### 함수 정의: 게시글 내용, 게시글 작성 일시, 좋아요 수, 해시태그 가져오기

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    try:
        content = soup.select('div._a9zs > span')[0].text
    except:
        content = ' '
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade > span')
    except:
        like = 0
    
    try:
        place =soup.select('div._aacl._aacn._aacu._aacy._aada._aade')[0].text
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data

#### 함수 정의: 다음 게시물로 이동

In [ ]:
def move_next(driver):
    right = driver.find_element_by_css_selector("div._aaqg._aaqh")
    right.click()
    time.sleep(3)

#### chrome driver 작동

* 코드가 실행되는 컴퓨터의 chrome 버전에 맞는 chrome driver를 다운받아 코드와 같은 경로에 저장해야 함

In [ ]:
driver =  webdriver.Chrome('chromedriver.exe')

driver.get('https://www.instagram.com')
time.sleep(3)

#### 로그인

* 개인의 인스타그램 아이디를 email 변수에, 비밀번호를 password 변수에 입력해야 함

In [ ]:
email = 'type_your_id' #'instagram ID'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'type_your_password' #'instagram PW'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(8)

#### 제공받을 트렌드의 키워드(검색어)를 입력

* 한글 검색어만 입력 가능

In [ ]:
word = input("한글 검색어를 입력하세요 : ")
while True:
    if isKoreanIncluded(word) == True:
        search_word = str(word)
        break
        
    else:
        word = input("한글이 아닙니다. 한글 검색어를 입력하세요 : ")
        
url = insta_searching(search_word)        
driver.get(url)
time.sleep(8)

#### 사용자가 입력한 검색 키워드의 검색 키워드를 찾을 수 있다면 크롤링 시작
* 찾을 수 없을 땐 새로운 검색어를 입력해야 함

In [ ]:
while True:
    if driver.find_elements_by_css_selector("div._aagw") == []:
        print("죄송합니다. 검색 결과를 찾을 수 없습니다.")
        word = input("새로운 검색어를 입력하세요 : ")
        search_word = str(word)
        url = insta_searching(search_word)
        driver.get(url)
        time.sleep(8)
    else:
        break

select_first(driver)

#### 1000개의 게시글을 크롤링

In [ ]:
results = []

target = 1000

for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        try:
            move_next(driver)
        except:
            break

    except:
        time.sleep(2)
        try:
            move_next(driver)
        except:
            break

print(results[:2])

#### 크롤링 결과 --> pandas DataFrame

In [ ]:
result_df = pd.DataFrame(results)
result_df.columns = ['content','date','like','place','tags']

#### 분석에 배제될 단어 입력
* 엔터 입력 시 종료됨

In [ ]:
exclude = []
while True:
    ex = input("분석에 배제될 단어를 입력하세요(엔터시 종료): ")
    if ex == '':
        break
    else:
        exclude.append(ex)

____

# 게시글 분석

## 1. 전처리

In [ ]:
docs = result_df
N = len(docs)

In [ ]:
docs

#### 좋아요 수 전처리

In [ ]:
for i in range(N):
    try:
        num = int(docs.iloc[i,2][0].getText())
    except:
        num = 0
    docs.iloc[i,2] = num

In [ ]:
docs.head(10)

#### 광고성 해시태그 제거

In [ ]:
remov = []
ad = ["#광고", "#유료광고", "#제품증정광고", "#제품협찬", "#협찬"]

for i in range(N):
    for j in range(len(docs.iloc[i][4])):
        if docs.iloc[i][4][j] in ad:
            if i not in remov:
                remov.append(i)

docs = docs.drop(remov)

In [ ]:
docs.head(10)

#### 도배 게시글 제거

In [ ]:
M = len(docs)

for i in range(M):
    docs.iloc[i][4] = tuple(docs.iloc[i][4])

In [ ]:
docs = docs.drop_duplicates(keep = 'first')

In [ ]:
L = len(docs)

for i in range(L):
    docs.iloc[i][4] = list(docs.iloc[i][4])

In [ ]:
docs.head(10)

#### 이모지 제거
* Co: 이모지 제거된 1차원 리스트
* cco : 이모지 제거 및 검색 키워드 유지된 2차원 리스트

In [ ]:
def rmEmoji(input):
    return input.encode('utf-8', 'ignore').decode('utf-8')

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)

only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"
                           "]+", flags=re.UNICODE)

In [ ]:
emoji_list=['☀️','•','❣️','✨','⭐','☕️','❤️','@','☁','♥️','✔','♀️','❣','⊙','×','✅','▪️','⬆️','⚡️','✋','❌','^','⭕️','❗️','▫','〰️','✈️','★','▶','➰']

In [ ]:
co = []
Co = []

for i in range(L):
    sen = ''
    content = docs.iloc[i,0].split()
    for word in content:
        rm_emj = rmEmoji(word)
        rm_ = emoji_pattern.sub(r'', rm_emj)
        rm_again = only_BMP_pattern.sub(r'', rm_)
        if '#' not in rm_again:
            sen += (rm_again+' ')
    co.append(sen) 

for i in range(len(co)):
    content = co[i].split()
    sen = ''
    for word in content:
        for emoji in emoji_list:
            if emoji in word:
                word = word.replace(emoji,"")
        sen += (word+' ')     
    Co.append(sen)

In [ ]:
remove_word = {search_word}

cco=[]

for k in range(len(Co)):
    temp = Co[k].split()
    cco.append([i for i in temp])

#### 사용자가 분석에 배제되길 원하는 단어를 게시글에서 제거
* Cco: 분석에 배제되는 단어가 제거된 2차원 리스트

In [ ]:
remove_word2 = set()
for i in range(len(cco)):
    for j in range(len(cco[i])):
        for k in range(len(exclude)):
            if exclude[k] in cco[i][j]:
                remove_word2.update([cco[i][j]])
                
Cco =[]
for i in range(len(cco)):
    temp = []
    for j in range(len(cco[i])):
        if cco[i][j] not in remove_word2:
            temp.append(cco[i][j])
    Cco.append(temp) 

#### 한글 명사만 추출
* 한글 처리기로 KoNLPy의 Okt 사용
* pos: 검색 키워드의 위치가 저장됨
* cc_kor: 한글만 추출 & 검색 키워드 유지된 2차원 리스트
* corp: 한글 명사만 추출 & 검색 키워드 제거된 2차원 리스트

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
cc_kor = []
for i in range(len(Cco)):
    temp = []
    for word in Cco[i]:
        if isKoreanIncluded(word) == True:      
            temp.append(word)
    if len(temp) != 0:
        cc_kor.append(temp)

검색 키워드가 okt.nouns에 의해 분리될 수 있어, nouns 사용 전 미리 키워드를 제거하는 작업. 추후에 복구됨

In [ ]:
nosearch_cc_kor =[]
pos = []

for i in range(len(cc_kor)):
    ttt = []
    for j in range(len(cc_kor[i])):
        if search_word not in cc_kor[i][j]:
            ttt.append(cc_kor[i][j])
        else:
            pos.append(i)
            
    nosearch_cc_kor.append(ttt)

In [ ]:
corp = []
for i in range(len(nosearch_cc_kor)):
    temp_kstr = ""
    for word in nosearch_cc_kor[i]:
        temp_kstr += (word+" ")
    new_kstr = okt.nouns(temp_kstr)
    corp.append(new_kstr)

#### 전처리 완료
* corpu: 전처리 & 키워드 제거된 2차원 리스트
* corpus: 전처리 & 키워드 제거된 1차원 리스트
* noremcorp: 전처리 & 검색 키워드 유지된 2차원 리스트
* Noremcorpus: 전처리 & 검색 키워드 유지된 1차원 리스트

명사 추출 후 검색 키워드가 다시 등장할 수 있으니 키워드 한 번 더 제거

In [ ]:
corpu = []
for i in range(len(corp)):
    temp = [i for i in corp[i] if i not in remove_word]
    corpu.append(temp)

In [ ]:
corpus =[]
for i in range(len(corpu)):
    for j in range(len(corpu[i])):
        corpus.append(corpu[i][j])

cosine similarity 계산을 위해 키워드 복구 진행

In [ ]:
noremcorp = []

for i in range(len(corp)):
    noremcorp.append(corp[i].copy())

In [ ]:
for i in range(len(pos)):
    noremcorp[pos[i]].append(search_word)

In [ ]:
Noremcorpus =[]
for i in range(len(noremcorp)):
    for j in range(len(noremcorp[i])):
        Noremcorpus.append(noremcorp[i][j])

---

## 2. TfidfVectorizer & TF-IDF 값 계산

### 상위 10개 단어 순위 제공

#### word embedding: TfidfVectorizer
* corpus(전처리 & 키워드 제거된 1차원 리스트)를 사용

In [ ]:
joined_corpus = ' '.join(corpus)
joined_corpus = joined_corpus.lower()

In [ ]:
tfidf_corpus = [joined_corpus]

In [ ]:
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(tfidf_corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(tfidf_corpus):
    tfidf_ = ((token, sp_matrix[i, word2id[token]]) for token in sent.split())

#### TF-IDF 값 계산
* sorted_tfidf: TF-IDF 값 내림차순 정렬

In [ ]:
tfidf = dict(tfidf_)
sorted_tfidf = sorted(tfidf.items(), key = lambda item: item[1], reverse = True)

In [ ]:
print("TF-IDF 값 순위")
print()

try:
    for i in range(10):
        print(i+1,": ",sorted_tfidf[i][0], round(sorted_tfidf[i][1],5))
except:
    print()

___

## 3. Word2Vec & cosine similarity

### 평균 이상의 유사도를 가진 단어들의 wordcloud 제공

#### word embedding: Word2Vec
* noremcorp(전처리 & 검색 키워드 유지된 2차원 리스트)를 사용

In [ ]:
model = Word2Vec(noremcorp, sg = 1, window = 5, min_count = 1)
model.init_sims(replace=True)

#### 검색 키워드와 매 게시글 단어 간 cosine similarity 및 similarity의 평균값 계산
* all_avg: cosine similarity 평균값

In [ ]:
count = 0
sum_value = 0

for i in range(len(noremcorp)):
    for j in range(len(noremcorp[i])):
        sum_value += model.wv.similarity(search_word, noremcorp[i][j])
        count += 1

all_avg = sum_value/count
print("average similarity value:", all_avg)

#### 평균 초과의 유사도 값을 갖는 게시글 단어만 추출
* similar_noremcorp: 평균 초과의 유사도를 갖는 단어가 저장된 1차원 리스트

In [ ]:
similar_noremcorp=[]

for i in range(len(noremcorp)):
    for j in range(len(noremcorp[i])):
        if model.wv.similarity(search_word, noremcorp[i][j]) > all_avg:
            similar_noremcorp.append(noremcorp[i][j])

#### 유사도 계산 후 키워드 제거
* similar_corpus: 평균 초과의 유사도 & 키워드 제거된 1차원 리스트

In [ ]:
similar_corpus = [i for i in similar_noremcorp if i not in remove_word]

#### 유사도 평균 초과 단어들의 wordcloud

In [ ]:
similar_best = ' '.join(similar_corpus)
similar_best = similar_best.lower()

In [ ]:
freq = FreqDist(similar_best.split())
wc = WordCloud(font_path='C:\\Windows\\Fonts\\malgun.ttf', width=3000, height=2000, background_color="white", random_state=0)
plt.imshow(wc.generate_from_frequencies(freq))
plt.axis("off")
plt.show()

___

## 4. Word2Vec & Topic Modeling

### 토픽 내 단어별 빈도수 및 토픽 별 wordcloud 제공

#### gensim 라이브러리로 LDA 객체 학습시키기

In [ ]:
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

os.environ['MALLET_HOME'] = 'C:/Mallet'
#c드라이브에 Mallet 폴더에 저장 및 환경변수 설정
id2word=corpora.Dictionary(noremcorp)
id2word.filter_extremes(no_below = 10)

#data word -> 게시글별로
 #corpus_list_to_list -> 해시태그별로 ?
texts = noremcorp #data word
corpus=[id2word.doc2bow(text) for text in texts]


#C:\Mallet 변수 값 MALLET_HOME
#os.environ.update({'MALLET_HOME':r'C:\Mallet'}) #Mallet path 지정
#C:/Users/me/Desktop/Mallet-202108/bin/mallet
#os.environ['MALLET_HOME']='C:/Users/me/Desktop/Mallet-202108/bin/mallet'

#지정한 폴더의 mallet 모듈 경로 쓰기 & gensim import
mallet_path='C:/Mallet/bin/mallet'
ldamallet=gensim.models.wrappers.LdaMallet(mallet_path,corpus=corpus, num_topics=4,id2word=id2word)

#### 토픽 수를 통해 coherence value를 계산 & 토픽 별 정렬한 coherence value 보이기
#### 함수 정의: mallet 모듈을 통한 토픽 format 정의
* compute_coherence_values 함수는 오래 걸릴 수 있다.

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()


def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=noremcorp, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values



# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=21, step=2)




limit=21; start=4; step=2;
x = range(start, limit, step)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1

def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    
    #이부분 수정
    #contents = pd.Series(texts)
    #sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    
    
    sent_topics_df = pd.concat([sent_topics_df, docs['tags']], axis=1)
    #,docs['timestamp'],docs['tweet_url'],docs['screen_name'],docs['label'] 얘네 지움
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=cc_kor)

# Format
df_topic_tweet = df_topic_sents_keywords.reset_index()
df_topic_tweet.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Tags']
#'Timestamp', 'Tweet_url','Screen_name','label'얘네 지움

# Show각 문서에 대한 토픽
#df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
#df_topic_tweet


# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "Tags","Num_Documents", "Perc_Documents"]
#, "timestamp", "tweet_url","screen_name","label"얘네 지움
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
#lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]

In [ ]:
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))

#### 함수 정의: 적절한 토픽 수 찾기

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=noremcorp, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

#### 토픽 수에 대한 coherence value를 그래프로 제공

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=6, limit=41, step=2)

In [ ]:
limit=41; start=6; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

#### 토픽 모델링 및 coherence value를 csv 파일로 저장

In [ ]:
lda_inform.to_csv ("C:/Users/me/Desktop/lda_inform.csv", index = None)
lda_inform

#### LDA 모델의 coherence score 출력
* 높을수록 좋다

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=noremcorp, dictionary=id2word,coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()      
print('\nCoherence Score: ' , coherence_ldamallet)                                          

#### 토픽 모델링 결과를 pyLDAvis로 시각화 및 html 파일로 저장
* 람다 값을 조절하여, 두 기준의 가중치를 변경하여 새로운 키워드를 도출 가능

In [ ]:
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model,corpus,id2word)
vis

In [ ]:
pyLDAvis.save_html(vis,'LDAMallet_TopicData_Visualization.html')

#### 토픽별 wordcloud 중 4개

In [ ]:
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0,
                   font_path='C:\\Windows\\Fonts\\malgun.ttf' )

topics = ldamallet.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

#### 모든 토픽의 wordcloud

In [ ]:
wc = WordCloud(background_color='white',
        font_path='C:\\Windows\\Fonts\\malgun.ttf')

plt.figure(figsize=(30,30))
for t in range(ldamallet.num_topics):
    plt.subplot(5,4,t+1)
    x = dict(ldamallet.show_topic(t,200))
    im = wc.generate_from_frequencies(x)
    plt.imshow(im)
    plt.axis("off")
    plt.title("Topic #" + str(t))

plt.savefig('LDA_wordcloud.png', bbox_inches='tight')

#### 토픽 별 단어 빈도수를 그래프화

In [ ]:
plt.rcParams["font.family"] = "NanumGothic"
topics = ldamallet.show_topics(formatted=False)
data_flat = [w for w_list in noremcorp for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    #ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 1); ax.set_ylim(0, 500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False,)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')
    
fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count of Topic Keywords', fontsize=22, y=1.05)    
plt.show()

___

# 해시태그 분석

## 1. 전처리

#### 해시태그만 추출

In [ ]:
hdocs = docs.iloc[:,[4]]
N = len(hdocs)

hdocs.head(5)

#### 해시태그의 '#' 제거

In [ ]:
for i in range(N):
    for j in range(len(hdocs.iloc[i,0])):
        hdocs.iloc[i,0][j] = hdocs.iloc[i,0][j].replace("#","")

In [ ]:
for i in range(N):       
    string = ' '.join([str(item) for item in hdocs.iloc[i,0]])
    hdocs.iloc[i,0] = string

In [ ]:
hdocs.head(5)

#### 이모지 제거
* co_: 이모지 제거된 1차원 리스트

In [ ]:
c_ = []
for i in range(N):
    arr = hdocs.iloc[i,0].split()
    c_ += arr

In [ ]:
co_ = []
for i in range(N):
    for word in c_:
        rm_emj = rmEmoji(word)
        rm_ = emoji_pattern.sub(r'', rm_emj)
        rm_again = only_BMP_pattern.sub(r'', rm_)
        if '#' not in rm_again:
            co_.append(rm_again) 

In [ ]:
for word in co_:
    for emoji in emoji_list:
        if emoji in word:
            pos = co_.index(word)
            co_[pos] = word.replace(emoji,"")
            
    if '' in co_:
        co_.remove('')

#### 검색 키워드 제거
* cc: 검색 키워드 제거된 1차원 리스트

In [ ]:
remove_word = {search_word}

In [ ]:
cc = [i for i in co_ if i not in remove_word]

#### 소통용 해시태그 제거
* CC: 소통용 해시태그 제거된 1차원 리스트

In [ ]:
daily = ['좋반','좋아요','좋아요반사','좋아요테러','좋반테러','좋테','좋아요그램','선팔','맞팔','선팔하면맞팔','선팔맞팔','맞팔해요','선팔환영','맞팔환영','팔로미','팔로우','팔로우미','팔로','소통', '일상','데일리','디엠환영','디엠','인친']
CC = []
for i in range(len(cc)):
    if cc[i] not in daily:
        CC.append(cc[i])

#### 사용자가 분석에 제거되길 원하는 단어가 포함된 해시태그를 제거
* CC: 소통용 해시태그 & 사용자 지정 배제 해시태그 제거된 1차원 리스트

In [ ]:
remove_word3 = set()
for i in range(len(CC)):
    for j in range(len(exclude)):
        if exclude[j] in CC[i]:
            remove_word3.update([CC[i]])

In [ ]:
CC = [i for i in CC if i not in remove_word3]

#### 한글만 추출 및 전처리 완료
* cc_kor: 전처리 완료된 해시태그가 저장된 1차원 리스트

In [ ]:
cc_kor_ = []
for word in CC:
    if isKoreanIncluded(word) == True:      
        cc_kor_.append(word)
    if '' in cc_kor_:
        cc_kor_.remove('')

---

## 2. TfidfVectorizer & TF-IDF 값 계산

### 상위 10개 해시태그의 순위 제공

#### word embedding: TfidfVectorizer
* cc_kor(전처리 완료된 해시태그가 저장된 1차원 리스트)을 사용

In [ ]:
ccc = ' '.join(cc_kor_)
cc_ = ccc.lower()

In [ ]:
hashtag_corpus = [cc_]

In [ ]:
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(hashtag_corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(hashtag_corpus):
    tfidf_ = ((token, sp_matrix[i, word2id[token]]) for token in sent.split())

#### TF-IDF값 계산
* sorted_tfidf_: TF-IDF 값 내림차순 정렬

In [ ]:
tfidf = dict(tfidf_)
sorted_tfidf_ = sorted(tfidf.items(), key = lambda item: item[1], reverse = True)

In [ ]:
print("TF-IDF 값 순위")
print()
try:
    for i in range(10):
        print(i+1,": ",sorted_tfidf_[i][0], round(sorted_tfidf_[i][1],5))
except:
    print()

## 3. 해시태그 빈도수에 따른 wordcloud

word embedding 없이, 오직 해시태그 자체의 빈도수에 따른 wordcloud

In [ ]:
freq = FreqDist(tuple(cc_kor_))
wc = WordCloud(font_path='C:\\Windows\\Fonts\\malgun.ttf', width=3000, height=2000, background_color="white", random_state=0)
plt.imshow(wc.generate_from_frequencies(freq))
plt.axis("off")
plt.show()

___

# 부가기능: 좋아요 수에 따른 인기 게시물 순위

#### 모든 언어의 상위 10개 게시글 제공

In [ ]:
docs_like = docs.sort_values(by=docs.columns[2],ascending = False)

docs_like.head(10)

___

# 부가기능: 많이 언급된 인기 있는 장소

In [ ]:
place = docs.iloc[:,[3]]
N = len(place)

In [ ]:
place_list = []
for i in range(N):
    place_list.append(place.iloc[i,0])
    if '' in place_list:
        place_list.remove('')

In [ ]:
place_withcount = []
for i in place_list:
    found = False
    count_place = []
    count_place.append(i)
    count_place.append(place_list.count(i))
    for j in range(len(place_withcount)):
        if i == place_withcount[j][0]:
            found = True
    if found == False:
        place_withcount.append(count_place)

In [ ]:
place_withcount.sort(key = lambda x:-x[1])

In [ ]:
for i in range(10):
    plt.bar(i+1, place_withcount[i][1])
plt.xticks(range(len(place_withcount)+1), range(len(place_withcount)+1))
plt.show()

for i in range(10):
    print(i+1,": ",place_withcount[i][0], "(", place_withcount[i][1], "개)")